## Location Database for interfacing data

***NOTE: The resulting CSV here includes latitude and longitude data of the counties associated with the fips codes but does not have the specific latitude and longitude data for the cities and zip codes.  This is a specific reference for a visual API and not intended to be used to associate the lattitude and longitude numbers with the cities or zip codes.***

This dataframe and associated CSV connects all of the fips ids, zip codes, county and atate names, and the latitude/longitude data in the United States.

In [1]:
import numpy as np
import pandas as pd


In [9]:
path_to_home = '../'
path_to_data = path_to_home+'data/BIG-QUERY/fips_zips/data/'
filename = path_to_data+'zip_county_fips_2018_03.csv'

We set the paths above so that we can easily change this file later if we move it to a different directory. Now import the data files.  The first one is called 'datapackage.json'. Later, use the same format but reset the 'filename' variable.

In [10]:
fips_df = pd.read_csv(filename)
fips_df.head()

zip  stcountyfp           city state      countyname classfp
0  36091        1001        Verbena    AL  Autauga County      H1
1  36758        1001  Plantersville    AL  Autauga County      H1
2  36006        1001    Billingsley    AL  Autauga County      H1
3  36067        1001     Prattville    AL  Autauga County      H1
4  36701        1001          Selma    AL  Autauga County      H1

In [11]:
fips_df.shape

(53962, 6)

In [31]:
fips_df.rename(columns=({'stcountyfp': 'fips', 'countyname': 'county_name'}), inplace=True)

In [32]:
fips_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53962 entries, 0 to 53961
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   zip          53962 non-null  int64 
 1   fips         53962 non-null  int64 
 2   city         53962 non-null  object
 3   state        53962 non-null  object
 4   county_name  53962 non-null  object
 5   classfp      53962 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.5+ MB


## Great. Now try searching this new dataframe for some known zip codes.

In [19]:
# testzip = 3054  # Merrimack, New Hampshire
# testzip = 95060  # Santa Cruz, California
# testzip = 94558  # Napa, California
## In the case of Napa, this search yielded three results, three 
## different fips codes (stcountyfp), and three different counties
testzip = 94115  # San Francisco, California

fips_df[fips_df.zip == testzip]

zip  stcountyfp           city state            countyname classfp
4869  94115        6075  San francisco    CA  San Francisco County      H6

As noted in the above cell, there are some zip codes that will yield multiple fips codes and multiple counties. In those cases, our searching algorithm can call the first one that it finds, or the lowest index.

## Now pull in a dataset that contains latitude/longitude data and at least one of the unique fields above

In [20]:
path_to_home = '../'
path_to_data = path_to_home+'data/BIG-QUERY/'
filename = path_to_data+'Covid_and_Pop_by_County.csv'

In [22]:
lats_df = pd.read_csv(filename)
lats_df.head()

Unnamed: 0  population     county_name state_name  \
0  Autauga County, Alabama     55869.0  Autauga County    Alabama   
1  Baldwin County, Alabama    223234.0  Baldwin County    Alabama   
2  Barbour County, Alabama     24686.0  Barbour County    Alabama   
3     Bibb County, Alabama     22394.0     Bibb County    Alabama   
4   Blount County, Alabama     57826.0   Blount County    Alabama   

   cumulative_confirmed  cumulative_deceased   latitude  longitude  
0                1015.0                 21.0  32.536667 -86.648333  
1                3101.0                 22.0  30.733056 -87.720278  
2                 598.0                  5.0  31.865833 -85.396111  
3                 363.0                  2.0  32.997778 -87.126389  
4                 767.0                  3.0  33.966667 -86.583333

For this dataset we only need location data, so drop the other columns

In [34]:
lats_df.drop(columns=(['Unnamed: 0', 'cumulative_confirmed', 'cumulative_deceased']), inplace=True)

In [35]:
lats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3020 entries, 0 to 3019
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   population   3020 non-null   float64
 1   county_name  3020 non-null   object 
 2   state_name   3020 non-null   object 
 3   latitude     3020 non-null   float64
 4   longitude    3020 non-null   float64
 5   state        3020 non-null   object 
dtypes: float64(3), object(3)
memory usage: 141.7+ KB


In [23]:
lats_df.state_name.value_counts()

Texas                   250
Georgia                 159
Kentucky                120
Missouri                109
Kansas                  103
Illinois                101
North Carolina          100
Iowa                     99
Tennessee                95
Virginia                 95
Indiana                  91
Ohio                     88
Minnesota                86
Nebraska                 84
Mississippi              82
Michigan                 81
Oklahoma                 77
Arkansas                 74
Wisconsin                71
Pennsylvania             67
Alabama                  66
Florida                  65
South Dakota             65
Colorado                 63
New York                 61
California               58
Louisiana                55
West Virginia            55
North Dakota             53
Montana                  51
South Carolina           46
Idaho                    42
Washington               39
Oregon                   35
New Mexico               32
Utah                

## Hardcode a dictionary with the states names and their abbreviateions to map on to the dataframe

In [27]:
state_abbr = {
    'Texas': 'TX',
    'Georgia': 'GA',
    'Kentucky': 'KY',
    'Missouri': 'MO',
    'Kansas': 'KS',
    'Illinois': 'IL',
    'North Carolina': 'NC',
    'Iowa': 'IA',
    'Tennessee': 'TN',
    'Virginia': 'VA',
    'Indiana': 'IN',
    'Ohio': 'OH',
    'Minnesota': 'MN',
    'Nebraska': 'NE',
    'Mississippi': 'MS',
    'Michigan': 'MI',
    'Oklahoma': 'OK',
    'Arkansas': 'AR',
    'Wisconsin': 'WI',
    'Pennsylvania': 'PA',
    'Alabama': 'AL',
    'Florida': 'FL',
    'South Dakota': 'SD',
    'Colorado': 'CO',
    'New York': 'NY',
    'California': 'CA',
    'Louisiana': 'LA',
    'West Virginia': 'WV',
    'North Dakota': 'ND',
    'Montana': 'MT',
    'South Carolina': 'SC',
    'Idaho': 'ID',
    'Washington': 'WA',
    'Oregon': 'OR',
    'New Mexico': 'NM',
    'Utah': 'UT',
    'Wyoming': 'WY',
    'Maryland': 'MD',
    'New Jersey': 'NJ',
    'Nevada': 'NV',
    'Maine': 'ME',
    'Arizona': 'AZ',
    'Massachusetts': 'MA',
    'Vermont': 'VT',
    'New Hampshire':'NH',
    'Connecticut':'CT',
    'Rhode Island':'RI',
    'Hawaii':'HI',
    'Delaware': 'DE',
    'Alaska': 'AK',
    'District of Columbia': 'DC'
}

In [28]:
lats_df['state'] = lats_df['state_name'].map(state_abbr)

In [29]:
lats_df['state']

0       AL
1       AL
2       AL
3       AL
4       AL
        ..
3015    WY
3016    WY
3017    WY
3018    WY
3019    WY
Name: state, Length: 3020, dtype: object

## Merge the two dataframes so we have latitude and logitude data in the fips_df 

In [56]:
#df1.merge(df2, left_on='lkey', right_on='rkey') --from the pandas documentation
fips_df = fips_df.merge(lats_df, left_on=['state', 'county_name'], right_on=['state', 'county_name'])

In [57]:
fips_df.head()

zip  fips           city state     county_name classfp  population  \
0  36091  1001        Verbena    AL  Autauga County      H1     55869.0   
1  36758  1001  Plantersville    AL  Autauga County      H1     55869.0   
2  36006  1001    Billingsley    AL  Autauga County      H1     55869.0   
3  36067  1001     Prattville    AL  Autauga County      H1     55869.0   
4  36701  1001          Selma    AL  Autauga County      H1     55869.0   

  state_name   latitude  longitude  
0    Alabama  32.536667 -86.648333  
1    Alabama  32.536667 -86.648333  
2    Alabama  32.536667 -86.648333  
3    Alabama  32.536667 -86.648333  
4    Alabama  32.536667 -86.648333

In [58]:
fips_df.drop(columns=['classfp', 'population'], inplace=True)

In [59]:
fips_df.to_csv(path_to_home+'data/BIG-QUERY/fips_lats_cities.csv', index=False)